In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
train_images=train_images
test_images=test_images

In [5]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [6]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [7]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Myfiles\Apps\envs\tensor\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [9]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [10]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.8196666836738586

Best val_accuracy So Far: 0.8840000033378601
Total elapsed time: 00h 01m 40s
INFO:tensorflow:Oracle triggered exit


In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        27696     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 80)                2211920   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 2,241,066
Trainable params: 2,241,066
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1715 - accuracy: 0.9358 - val_loss: 0.3612 - val_accuracy: 0.8870
Epoch 5/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1357 - accuracy: 0.9503 - val_loss: 0.4209 - val_accuracy: 0.8850
Epoch 6/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1147 - accuracy: 0.9570 - val_loss: 0.4796 - val_accuracy: 0.8815
Epoch 7/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0936 - accuracy: 0.9666 - val_loss: 0.5168 - val_accuracy: 0.8872
Epoch 8/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0780 - accuracy: 0.9729 - val_loss: 0.5406 - val_accuracy: 0.8777
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0706 - accuracy: 0.9764 - val_loss: 0.6683 - val_accuracy: 0.8912
Epoch 10/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0613 - accuracy: 0.9792 - val_loss: 0.6562 - val_accuracy